# 8- MÉTRICAS

 La rápida expansión de NILM y el desarrollo de diferentes algoritmos, han hecho que sea esencial proporcionar una evaluación de rendimiento mediante el uso de métricas de desempeño. Las métricas de evaluación, comparan los resultados de la desagregaciónn (predicciones) de los modelos entrenados con los datos del set de validación (mediciones reales de cada proceso). NILMTK cuenta con el cálculo de métricas de evaluación mediante el uso del MeterGroup para la validación de los resultados mediante el set de validación 
 Vamos ahora analizar en nuestro dataset diferentes metricas como son FEAC,F1, EAE ,MNEAP y RMSE  que  resumimos a continuacion en el siguinte cuadro:

![metricas](metricas.jpg)

En primer lugar cargamos las ibrerías y realizamos otros ajustes:

In [59]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import warnings
warnings. filterwarnings("ignore")
plt.style.use('bmh')

from nilmtk import TimeFrame, MeterGroup, HDFDataStore, DataSet
from nilmtk.metrics import fraction_energy_assigned_correctly, mean_normalized_error_power
from nilmtk.metrics import rms_error_power, f1_score, error_in_assigned_energy

data_dir='./data/'

### Métricas individuales del modelo

In [60]:
#Librerías
from nilmtk.metrics import f1_score
from nilmtk.metrics import error_in_assigned_energy
from nilmtk.metrics import mean_normalized_error_power
from nilmtk.metrics import rms_error_power
from nilmtk.metrics import fraction_energy_assigned_correctly

prueba

In [61]:
labels = pd.DataFrame({'Instances': list(range(1,13)),
                       'Meters': ['Medidor Princ.','M2', 'M.Fridge', 'M.AA', 'M.AA', 'M.Washing machine.','Computer','Clothes iron','unknown','tv','wet appliance','motor']})

#Set para validación
valid = DataSet('iawe.h5')



valid.set_window(start="2013-07-24 05:30:00", end="2013-08-24 05:30:00")
valid_elec = valid.buildings[1].elec

print('Medidor de cabecera para desagregación:')
valid_Main = valid_elec.mains()
print(valid_Main)

print('\nDispositivos para validación de entrenamiento:')
valid_Devices = valid_elec.select(instance=[3,4,5,6,7,8,9,10,11])
print(valid_Devices)

Medidor de cabecera para desagregación:
MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='iAWE', site_meter, appliances=[])
)

Dispositivos para validación de entrenamiento:
MeterGroup(meters=
  ElecMeter(instance=3, building=1, dataset='iAWE', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=4, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=5, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=2)])
  ElecMeter(instance=6, building=1, dataset='iAWE', appliances=[Appliance(type='washing machine', instance=1)])
  ElecMeter(instance=7, building=1, dataset='iAWE', appliances=[Appliance(type='computer', instance=1)])
  ElecMeter(instance=8, building=1, dataset='iAWE', appliances=[Appliance(type='clothes iron', instance=1)])
  ElecMeter(instance=9, building=1, dataset='iAWE', appliance

In [62]:
labels = pd.DataFrame({'Instances': list(range(1,13)),
                       'Meters': ['Medidor Princ.','M2', 'M.Fridge', 'M.AA', 'M.AA', 'M.Washing machine.','Computer','Clothes iron','unknown','tv','wet appliance','motor']})

#Set para validación
valid = DataSet('iawe.h5')
valid.set_window(start="2013-07-24 05:30:00", end="2013-08-24 05:30:00")
valid_elec = valid.buildings[1].elec

print('Medidor de cabecera para desagregación:')
valid_Main = valid_elec.mains()
print(valid_Main)

print('\nDispositivos para validación de entrenamiento:')
valid_Devices = valid_elec.select(instance=[3,4,5,6,7,8,9,10,11])
print(valid_Devices)

Medidor de cabecera para desagregación:
MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='iAWE', site_meter, appliances=[])
)

Dispositivos para validación de entrenamiento:
MeterGroup(meters=
  ElecMeter(instance=3, building=1, dataset='iAWE', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=4, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=5, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=2)])
  ElecMeter(instance=6, building=1, dataset='iAWE', appliances=[Appliance(type='washing machine', instance=1)])
  ElecMeter(instance=7, building=1, dataset='iAWE', appliances=[Appliance(type='computer', instance=1)])
  ElecMeter(instance=8, building=1, dataset='iAWE', appliances=[Appliance(type='clothes iron', instance=1)])
  ElecMeter(instance=9, building=1, dataset='iAWE', appliance

In [63]:
#Datos de Dispositivos: predicción
# Se carga los resultados del mejor modelo
index = list(labels['Meters'][3:12])

dataset=DataSet(data_dir+'NILM/Results/CO_10min_first_pred.h5')

#dataset=DataSet(data_dir+'NILM/Results/CO_1s_mean_pred.h5')
predictions=dataset.buildings[1].elec.submeters()      


# Initialize empty lists or DataFrames before the loop
e = []
eae = pd.DataFrame()
mneap = pd.DataFrame()
rmse = pd.DataFrame()
met = []



f1['F1'] = f1_score(predictions, valid_Devices)

EAE = error_in_assigned_energy(predictions, valid_Devices)
for x in range(len(EAE)):
    e.append(EAE.iloc[x][0])

# Assuming `mean_normalized_error_power` and `rms_error_power` functions are defined elsewhere
eae['EAE'] = e
mneap['MNEAP'] = mean_normalized_error_power(predictions, valid_Devices)
rmse['RMSE'] = rms_error_power(predictions, valid_Devices)

# Assuming `index` is a list of indices
met.append(
    pd.concat([
        pd.DataFrame(f1),
        pd.DataFrame(eae, index=valid_Devices.instance()),
        pd.DataFrame(mneap, index=valid_Devices.instance()),  # Assuming `mean_normalized_error_power` returns data with the same index
        pd.DataFrame(rmse, index=valid_Devices.instance())  # Assuming `rms_error_power` returns data with the same index
    ], axis=1)
)

dataset.store.close()       
print('\n\nDone!')
metrics=met[0].transpose().round(2)#sirve para 0 y 1
metrics





Done!


,3,4,5,6,7,8,9,10,11
F1,0.59,1.00,0.97,0.72,NaN,NaN,NaN,NaN,NaN
EAE,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MNEAP,0.89,0.14,0.70,0.46,NaN,NaN,NaN,NaN,NaN
RMSE,68.54,481.86,848.53,135.60,NaN,NaN,NaN,NaN,NaN


## Métrica F1

Esta métrica mide la precisión o asertividad del modelo entrenado mediante el cálculo de la media armónica utilizando las métricas Precision y Recall. Estas métricas son calculadas internamente por NILMTK mediante la matriz de confusión, vease la ecuación de la figura siguiente.

![metricas](metricas.jpg)

Precision se entiende como la fracción de cortes de tiempo en los que se predijo correctamente que un proceso estaba activo, pero realmente se encontraba apagado. Por lo contrario, Recall hace referencia a la fracción de cortes de tiempo en los que se predijo correctamente que el dispositivo estaba encendido y realmente se encontraba encendido .

In [65]:
pd.DataFrame(metrics.loc['F1']).transpose()*100

,3,4,5,6,7,8,9,10,11
F1,59.0,100.0,97.0,72.0,NaN,NaN,NaN,NaN,NaN


## Métrica EAE

La ecuación de la figura representa la diferencia entre la energía total asignada yˆt(n) energía total consumida yt(n) por el proceso n de la industria en kWh 

![metricas](metricas.jpg)

In [66]:
round(pd.DataFrame(metrics.loc['EAE']).transpose(),2)

,3,4,5,6,7,8,9,10,11
EAE,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
predictions_Devices=elec_CO_30s_first.submeters()
e1=predictions_Devices[5].total_energy()[0]
e2=valid_Devices[5].total_energy()[0]
eae=round(abs(e1-e2),2)

pd.DataFrame([e1,e2,eae], columns=['kWh'], index=['Energy_pred','Energy_GT','EAE'])

,kWh
Energy_pred,46.685492
Energy_GT,4.958180
EAE,41.730000


## Métrica MNEAP

Este indicador que representa la ecuacion de la imagen  consta de la diferencia entre la potencia asignada yˆt(n) y la potencia real yt(n) de cada proceso de la industria, por cada intervalo de tiempo t. Después de sumar todas las diferencias de potencia, el resultado es normalizado por el consumo total de la energía del proceso 

![metricas](metricas.jpg)

In [68]:
round(pd.DataFrame(metrics.loc['MNEAP']).transpose(),2)

,3,4,5,6,7,8,9,10,11
MNEAP,0.89,0.14,0.7,0.46,NaN,NaN,NaN,NaN,NaN


## Métrica RMSE

Esta métrica de la ecuación de la figura es muy utilizada en la comparaciónn de diversos conjuntos de datos, consiste en el error cuadrático medio de la diferencia entre la potencia asignada yˆt(n) y la potencia real yt(n) del proceso de los applicativos , en cada intervalo de tiempo t .

![metricas](metricas.jpg)

In [69]:
round(pd.DataFrame(metrics.loc['RMSE']).transpose(),1)

,3,4,5,6,7,8,9,10,11
RMSE,68.5,481.9,848.5,135.6,NaN,NaN,NaN,NaN,NaN


## Resultados de todos los modelos

In [100]:
from nilmtk.metrics import fraction_energy_assigned_correctly, f1_score, error_in_assigned_energy
from nilmtk.metrics import mean_normalized_error_power, rms_error_power 
periods = {'10s':10 , '30s':30, '60s':60, '5min':60*5, '10min':60*10}
#Datos de Dispositivos: predicción
# Se carga los resultados del mejor modelo


dir_predictions = data_dir +'NILM/Results/'
m = {}
classifiers = ['CO', 'FHMM']
methods = ['first', 'mean', 'median']
#index = list(labels['Meters'][3:12])



# Extracting meter labels using iloc
#index = labels['Meters'].iloc[3:12].tolist()


for p_name, sp in periods.items():
    met = []
    for method in methods:
        for clf_name in classifiers:
            print('\nCalculando Metricas: '+ clf_name + '_' + p_name + '_' + method)  
           
            #Datos de Dispositivos: predicción
            input_dir = dir_predictions + clf_name + '_' + p_name + '_' + method +'_pred.h5'
            dataset = DataSet(input_dir)
            predictions=dataset.buildings[1].elec.submeters()       
            
         
            # Initialize empty lists or DataFrames before the loop
            e = []
            eae = pd.DataFrame()
            mneap = pd.DataFrame()
            rmse = pd.DataFrame()
            met = []
            f1['F1'] = f1_score(predictions, valid_Devices)
            EAE = error_in_assigned_energy(predictions, valid_Devices)
            for x in range(len(EAE)):
             e.append(EAE.iloc[x][0])
            # Assuming `mean_normalized_error_power` and `rms_error_power` functions are defined elsewhere
            eae['EAE'] = e
            mneap['MNEAP'] = mean_normalized_error_power(predictions, valid_Devices)
            rmse['RMSE'] = rms_error_power(predictions, valid_Devices)
            # Assuming `index` is a list of indices
            met.append(
                   pd.concat([
                   pd.DataFrame(f1),
                   pd.DataFrame(eae, index=valid_Devices.instance()),
                   pd.DataFrame(mneap, index=valid_Devices.instance()),  # Assuming `mean_normalized_error_power` returns data with the same index
                   pd.DataFrame(rmse, index=valid_Devices.instance())  # Assuming `rms_error_power` returns data with the same index
                   ], axis=1).set_index(pd.Index(index))
                     )
            metrics=met[0].transpose().round(2)#sirve para 0 y 1
            print (metrics)

            dataset.store.close()       
    m[p_name] = met

##Guardamos resultados
f = open(data_dir +'NILM/metrics.txt',"w")
f.write( str(m) )
f.close()
print('\n\nDone!')



Calculando Metricas: CO_10s_first
        M.AA    M.AA  M.Washing machine.  Computer  Clothes iron  unknown  tv  \
F1      0.63    0.97                0.93      0.87           NaN      NaN NaN   
EAE     0.00     NaN                 NaN       NaN           NaN      NaN NaN   
MNEAP   0.83    0.22                0.37      0.74           NaN      NaN NaN   
RMSE   66.81  720.42              747.95    183.36           NaN      NaN NaN   

       wet appliance  motor  
F1               NaN    NaN  
EAE              NaN    NaN  
MNEAP            NaN    NaN  
RMSE             NaN    NaN  

Calculando Metricas: FHMM_10s_first
        M.AA    M.AA  M.Washing machine.  Computer  Clothes iron  unknown  tv  \
F1      0.76    1.00                1.00      0.74           NaN      NaN NaN   
EAE     0.00     NaN                 NaN       NaN           NaN      NaN NaN   
MNEAP   0.56    0.10                0.36      0.48           NaN      NaN NaN   
RMSE   51.61  412.82              730.79    128.

## Otras Metricas: métricas Promedio

In [90]:
import pandas as pd

m_mean = {}

cols = ['F1', 'EAE', 'MNEAP', 'RMSE']
labels = ['CO-first', 'CO-mean', 'CO-median', 'FHMM-first', 'FHMM-mean', 'FHMM-median']
per = ['10s', '30s', '60s', '5min', '10min']

for c in cols:
    data = []
    for i in range(len(per)):
        try:
            # Try to access data for each time period and construct a row
            row = [m[per[i]][j][c].mean() for j in range(len(labels))]
        except IndexError:
            # If index out of range error occurs, fill the row with NaNs
            row = [float('nan')] * len(labels)
        data.append(row)

    # Construct the DataFrame using the data
    m_mean[c] = pd.DataFrame(data, columns=labels, index=per).transpose()

m_mean


{'F1':              10s  30s  60s  5min  10min
 CO-first     NaN  NaN  NaN   NaN    NaN
 CO-mean      NaN  NaN  NaN   NaN    NaN
 CO-median    NaN  NaN  NaN   NaN    NaN
 FHMM-first   NaN  NaN  NaN   NaN    NaN
 FHMM-mean    NaN  NaN  NaN   NaN    NaN
 FHMM-median  NaN  NaN  NaN   NaN    NaN,
 'EAE':              10s  30s  60s  5min  10min
 CO-first     NaN  NaN  NaN   NaN    NaN
 CO-mean      NaN  NaN  NaN   NaN    NaN
 CO-median    NaN  NaN  NaN   NaN    NaN
 FHMM-first   NaN  NaN  NaN   NaN    NaN
 FHMM-mean    NaN  NaN  NaN   NaN    NaN
 FHMM-median  NaN  NaN  NaN   NaN    NaN,
 'MNEAP':              10s  30s  60s  5min  10min
 CO-first     NaN  NaN  NaN   NaN    NaN
 CO-mean      NaN  NaN  NaN   NaN    NaN
 CO-median    NaN  NaN  NaN   NaN    NaN
 FHMM-first   NaN  NaN  NaN   NaN    NaN
 FHMM-mean    NaN  NaN  NaN   NaN    NaN
 FHMM-median  NaN  NaN  NaN   NaN    NaN,
 'RMSE':              10s  30s  60s  5min  10min
 CO-first     NaN  NaN  NaN   NaN    NaN
 CO-mean      NaN  NaN  